In [111]:
## Importación de librerias necesarias

import pandas as pd
import time
import json
import requests
import os
import pandas as pd
import snowflake.connector as sf
import pathlib

##Definición conexión snowflake
con = sf.connect(
user= 'hamilton.pacanchique@rappi.com',
account= 'hg51401',
warehouse='PAGOS_ANALYSTS',
database='FIVETRAN',
schema='PUBLIC',
authenticator='externalbrowser'
)

frequency = input('Ingrese la frecuencia: ')
country = input('Ingrese las iniciales del país: ')
frequency= int(frequency)


## Definición de función Snow extrae y pasa a pandas  
def sf_query (query): 
    PATH = pathlib.Path().parent.resolve()                                                                 ## Trae la información de la libreria donde está el archivo
    cur = con.cursor()
    cur.execute(open(str(PATH.joinpath(query)), "r").read(), _show_progress_bar= True)                     ## Trae el archivo pasandole el nombre y la extensión   
    result = cur.fetchall()
    colnames = [desc[0] for desc in cur.description]
    df = pd.DataFrame(result) 
    df.columns = colnames
    return df


######## Change Environment

environments = {
    'MX': 'https://services.mxgrability.rappi.com',
    'PE': 'https://services.rappi.pe',
    'AR': 'https://services.rappi.com.ar',
    'CR': 'https://services.rappi.co.cr',
    'CO': 'https://services.rappi.com',
    'CL': 'https://services.rappi.cl',
    'UY': 'https://services.rappi.com.uy',
    'EC': 'https://services.rappi.com.ec',
    'BR': 'https://services.rappi.com.br',
    'dev': 'http://microservices.dev.rappi.com'
}
    
#########Ejecución del query

print('Downloading Base')
df_base = sf_query('query.sql')                                                                              ##Se ejecuta el método creado anteriormente
contract_id = list(df_base['ID'])                                                                            ## Extracción del contract_id

#####
df = pd.DataFrame()
df['contract_id'] = contract_id
df['freq'] = frequency
lista = df.values.tolist()                                                                                 ## Cambia el formato de los campos a [[campo1,campo2]]
environment = environments[country]
print('Se modificará contract_id: '+(str(contract_id).replace('[','')).replace(']','') +' del país: '+country+ ' a la frecuencia: '+str(frequency) )  


Ingrese la frecuencia:  13
Ingrese las iniciales del país:  BR


Se modificará contract_id: 153504 del país: BR a la frecuencia: 13


In [58]:
for i in range(0,len(lista)):
    
    contract_id = lista[i][0]
    freq =  lista[i][1]
    print(contract_id)
    
    business_structure= {
          "frequency_type_id": freq,         
          "email": "hamilton.pacanchique@rappi.com"
        }
    
    headers = {
        'Content-Type': 'application/json'
    }
    
    body_data = json.dumps(business_structure)
    response = requests.put(f'{environment}/api/partner-payment/contract/update/{contract_id}',data=body_data, headers=headers)
    
    
    print(body_data)
    print('##############frequency##############')
    print('adjustment: '+ str(response.status_code))
    print(response.json())
    
time.sleep(1)

68577
{"frequency_type_id": 3, "email": "marcello.santos@rappi.com"}
##############frequency##############
adjustment: 200
{'id': 68577, 'responsible_id': 8450, 'start_date': '2020-06-19', 'end_date': '2021-12-31', 'frequency_type_id': 3, 'document_url': None, 'slots': [], 'conditions_type': [], 'stores': [], 'order_classification_selected_id': 1, 'order_classification_selected_name': '', 'conditions_type_by_default': {}, 'service_detail': None, 'active': True, 'exclusive': False}
20986
{"frequency_type_id": 3, "email": "marcello.santos@rappi.com"}
##############frequency##############
adjustment: 200
{'id': 20986, 'responsible_id': 10421, 'start_date': '2019-01-01', 'end_date': '2019-12-31', 'frequency_type_id': 3, 'document_url': None, 'slots': [], 'conditions_type': [], 'stores': [], 'order_classification_selected_id': 1, 'order_classification_selected_name': '', 'conditions_type_by_default': {}, 'service_detail': None, 'exclusive': None, 'active': True}
23139
{"frequency_type_id": 